In [ ]:
# LDA(Latent Dirichlet Allocation)
## 토픽은 각각 단어 분포를 가지고 있다고 가정한다.
### 1. n개의 토픽을 지정한다.
### 2. 무작위로 토픽의 혼합 비율을 설정
### 3. 단어들을 토픽에 할당
######## 현재 문서의 모든 단어를 토픽에 할당
######## 현재 문서에서 토픽 비율 업데이트
######## 모든 문서에서 토픽 비율 업데이트
### 4. 반복

# 1.데이터 불러오기

In [1]:
import pandas as pd

df = pd.read_csv("./data/appreply2.csv",index_col=0)

df.head()

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3


## 1.

# 2.텍스트 -> 숫자

### 1) 전처리

In [2]:
from kiwipiepy import Kiwi
import re

kiwi = Kiwi()

In [3]:
sample_code = kiwi.tokenize("안녕ㅇ하세요 저는 형태소 분석기 에요")
print(sample_code)


[Token(form='안녕', tag='IC', start=0, len=2), Token(form='ㅇ', tag='SW', start=2, len=1), Token(form='하', tag='VV', start=3, len=1), Token(form='세요', tag='EF', start=4, len=2), Token(form='저', tag='NP', start=7, len=1), Token(form='는', tag='JX', start=8, len=1), Token(form='형태소', tag='NNG', start=10, len=3), Token(form='분석기', tag='NNG', start=14, len=3), Token(form='이', tag='VCP', start=18, len=0), Token(form='에요', tag='EF', start=18, len=2)]


In [4]:
# Kiwipipey 폼사 태그 목록 : https://bab2min.github.io/kiwipiepy/v0.22.2/kr/#_11
# STEP1. 전처리 문장을 담을 빈 리스트를 준비한다. sent_list
sent_list=[]
for sent in df["text"]:
    # STEP2. df["text"]에서 문장을 하나씩 뺀다.
    # ,숫자 의미없어 뺀다
    # clean_sent = re.sub(r"[^1-9a-zA-Z가-핳\s]+","",sent)
    clean_sent = re.sub(r"[^a-zA-Z가-핳\s]+","",sent)
    # STEP3. 문장을 정규표현식을 이용해서 전처리 한다. clean_sent
    result = kiwi.tokenize(clean_sent)
    sub_list = []
    for x in result :
        word = x.form
        pos = x.tag
        
        # SETP4. clean_sent를 토크나이징 한다.(Kiwi 형태소 분석기 사용) result
        # 1) 조건 : 조사(J),어미(E),접미사(X) 는 포함하지 않는다.
        # 2) 조건추가 : 한 글자인 단어는 포함하지 않는다.
        if (pos[0] in ["J","E","X"]) or (len(word)<2): continue
            
        sub_list.append(word)
        
        # STEP5. result를 " "를 구분자로 하나의 문자열로 합친다. result_str
        result_str = " ".join(sub_list)
        #print(f"채우기 : {word}")
    
    # STEP6. result_str을 sent_list에 넣는다
    sent_list.append(result_str)
    #print(sent)
    #print("-"*50)
        

In [5]:
print(len(sent_list))
for old,new in zip(df["text"].tolist(),sent_list):
    print(f"OLD : {old}")
    print(f"NEW : {new}")
    print("-"*100)

1000
OLD : 배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,별점순 뿐만아니라 제가 주문하고자하는 메뉴를 특정해서 그 메뉴의 리뷰만 확인할 수 있는 기능이 있으면 좋을 것 같습니다. 메뉴검색 기능처럼 리뷰에서도 특정 메뉴 검색기능이 필요합니다. 주문수가 많지않은 메뉴의 리뷰를 보기 위해서 드래그하느라 시간이 많이 소요되는 비효율이 발생합니다. 긍정적으로 검토해주심 좋을 것 같습니다.
NEW : 배달 민족 주문 리뷰 자주 참고 가지 건의 사항 최신 점수 아니 주문 메뉴 특정 메뉴 리뷰 기능 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 필요 주문 메뉴 리뷰 드래그 시간 많이 소요 비율 발생 긍정 검토
----------------------------------------------------------------------------------------------------
OLD : 내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 기억은 있는데 뭘시켰고 맛있게 먹은게 맞는지 알고싶은데 찾기가 너무 어렵네요.. 먹던 메뉴도 검색가능했으면 좋겠어요! 예를 들면 곱창 을 치면 과거에 시킨 곱창목록이 뜨거나 혹은 가게 리뷰창에 내리뷰보기 가 있어도 좋을거같아요 :)
NEW : 주문 과거 목록 검색 기능 분명 가게 시키 기억 시키 맛있 너무 어렵 메뉴 검색 가능 곱창 과거 시키 곱창 목록 가게 리뷰 리뷰 보기
----------------------------------------------------------------------------------------------------
OLD : 검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스크롤하는데 자꾸만 왼쪽 오른쪽으로 넘어가서 전체 탭이나 포장 탭을 보게 되는데 정말 불편합니다. 검색 후 배달 탭, 포장 탭, B마트 탭 뭐든 하나를 선택했으면 좌우로 넘기는 건 안 되게 하시면 안되나요? 가끔 그러는 것

In [ ]:
# 나는 밥을 먹었다 -> (토그나이징) 나 / 는 / 밥 / 을 / 먹었다. -> 나 는 밥 을 먹었다.
# 단어들이 띄어쓰기를 구분자로 하여 하나의 문자열로 완성 되어야 한다.

### 2) 수치화

In [ ]:
# 라이브러리 설치
# uv add scikit-learn

In [ ]:
# CountVectorizer란?
# 문장에 단어가 몇개 들어가 있나요?
#      단어1    단어2    단어3
# 문장1 
# 문장2
# ...

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(
    max_df = 0.1,           # 전체 단어의 등장 비율이 P이상인 것만 사용
    min_df = 2,             # 이 단어가 적어도 n개 이상 있는것만 사용
    max_features = 1000,    # 최대 몇 개 까지 나타낼 것인가
    ngram_range=(1,2)       # 단어의 조합 설정(ex, 1개만 사용)
)
feat_vec = count_vectorizer.fit_transform(sent_list)
# 문장 - > 스페이스 기준으로 쪼갠다. -> 단어들이 나열
# 각 문장마다 단어가 몇번들어가 있는지ㅣ

In [7]:
df_vec = pd.DataFrame(
    feat_vec.toarray(),
    columns=count_vectorizer.get_feature_names_out()
)

# 결과확인
df_vec.head()

,cs,km,ui,ux,vip,가게 가게,가게 검색,가게 들어가,가게 리뷰,가게 메뉴,...,하지만,한두,한두 아니,한정,한참,할인,할인 쿠폰,함께,항목,항상
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
feat_vec.shape

(1000, 1000)

# 3. 토픽분석 시각화

In [44]:
# 라이브러리 설치
# uv add pyLDAvis

In [63]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5) # 몇개의 토픽으로 할까요?
lda.fit(feat_vec)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",5
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",10.0
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


In [64]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()

vis = pyLDAvis.lda_model.prepare(lda,feat_vec,count_vectorizer)
pyLDAvis.display(vis)

In [76]:
## 행 : 문장
## 열 : 그룹(n_component)
## 값 : 문장이 그룹에 포함될 확률
## ex : 문장1[0.1 0.8 0.1] -> 이 문장1은 그룹 2 이다
sent_topic = lda.transform(feat_vec)
print(sent_topic)

[[0.0060426  0.36272767 0.00610223 0.00595031 0.6191772 ]
 [0.01188656 0.0119546  0.01188194 0.31149796 0.65277894]
 [0.00694053 0.00702115 0.00693765 0.00693625 0.97216442]
 ...
 [0.00708318 0.42514855 0.55351759 0.00715397 0.00709672]
 [0.01143447 0.0113659  0.9543587  0.01162514 0.0112158 ]
 [0.00576629 0.97675138 0.00580548 0.00587735 0.0057995 ]]


In [77]:
import numpy as np

sent1= np.array([0.01169701, 0.01209843, 0.97620456])
print(sent1.sum())
print(sent1.max())    # 배열의 최대값
print(sent1.argmax()) # 배열에 최댓값이 있는 위치

1.0
0.97620456
2


In [ ]:
# 반복문
# STEP0. 빈리스트 2개를 준비한다.--topic_idx_list, topic_prob_list
# STEP1. sent_topic에서 요소 하나하나씩 뽑는다.[0.01169701, 0.01209843, 0.97620456]
# STEP2. 이문장을 어떤 그룹으로 분류 할건지 (argmax()) - topic_index
# STEP3. 이 문장이 분류 되었을때 그 확률 (max()) - topic_prob

In [78]:
# STEP0. 빈리스트 2개를 준비한다.--topic_idx_list, topic_prob_list
topic_idx_list = []
topic_prob_list = []


    # STEP1. sent_topic에서 요소 하나하나씩 뽑는다.[0.01169701, 0.01209843, 0.97620456]
for sent in sent_topic:
    # STEP2. 이문장을 어떤 그룹으로 분류 할건지 (argmax()) - topic_index
    topic_idx = sent.argmax()

    # STEP3. 이 문장이 분류 되었을때 그 확률 (max()) - topic_prob
    topic_prob = sent.max()

    # STEP4. topic_idx,topic_prob을 topic_idx_list,topic_prob_lsit에 담는다
    topic_idx_list.append(topic_idx)
    topic_prob_list.append(topic_prob)
    
    print(sent)
    print(f"{topic_idx}위치에 최대값 {topic_prob:.4f}이 있습니다.")
    print("-"*50)
    

[0.0060426  0.36272767 0.00610223 0.00595031 0.6191772 ]
4위치에 최대값 0.6192이 있습니다.
--------------------------------------------------
[0.01188656 0.0119546  0.01188194 0.31149796 0.65277894]
4위치에 최대값 0.6528이 있습니다.
--------------------------------------------------
[0.00694053 0.00702115 0.00693765 0.00693625 0.97216442]
4위치에 최대값 0.9722이 있습니다.
--------------------------------------------------
[0.00969886 0.84198203 0.00959889 0.12906273 0.00965749]
1위치에 최대값 0.8420이 있습니다.
--------------------------------------------------
[0.79160256 0.00722946 0.00720769 0.00723228 0.18672801]
0위치에 최대값 0.7916이 있습니다.
--------------------------------------------------
[0.00775815 0.42161533 0.09643177 0.00784657 0.46634818]
4위치에 최대값 0.4663이 있습니다.
--------------------------------------------------
[0.40548993 0.58575508 0.00294745 0.002906   0.00290154]
1위치에 최대값 0.5858이 있습니다.
--------------------------------------------------
[0.00507412 0.00507603 0.00515678 0.97959955 0.00509353]
3위치에 최대값 0.9796이 있습니다.
---

In [79]:
df["topic_no"] = topic_idx_list
df["topic_prob"] = topic_prob_list

In [73]:
df.head()

,text,score,topic_no,topic_prob
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4,2,0.976205
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5,2,0.957146
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1,2,0.976148
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2,0,0.957835
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3,1,0.651641


In [80]:
# 어떤토픽이 가장 많이 있나요? - value_count

df["topic_no"].value_counts() # 0

topic_no
1    276
2    215
4    206
3    202
0    101
Name: count, dtype: int64

In [ ]:
# 각 토픽별로 확률이 가장 높은 순서대로 보여 주세요(셀3개, 토픽 0,1,2,각각 프린트 ) - sort_values
## 토픽 0인 데이터 조회
df.loc[df["topic_no"]==0,:].sort_values(by=["topic_prob"],ascending=False).head(3)
df.loc[df["topic_no"]==0,:].sort_values(by=["topic_prob"], ascending=False)


#print(df.sort_values(by=["topic_no"],ascending=False).head(3))

,text,score,topic_no,topic_prob
799,주문현황에서 실시간 지도 움직이는거 생성해주세요. 배민원만 실시간 지도 움직이는거만...,3,0,0.990348
262,최악. 배달이 잘못되도 앱이 어떤식으로 구성되는지 내 주문 건 못찾는다고 주인이 화...,1,0,0.980799
745,도로명 주소로 주소저장해도 배달만 시키면 지번주소로 바뀜. CS문의하니 우리는 문제...,1,0,0.979579
121,"<< BH6H3MKS >> 어플 정말 잘 쓰고 있고, 쿠폰도 있고 요청사항도 가게랑...",5,0,0.976807
259,"어플 정말 잘 쓰고 있고, 쿠폰도 있고 요청사항도 가게랑 배달 관련으로 2개로 나뉘...",5,0,0.976807
...,...,...,...,...
468,가게에 남기는 메모가 가게별로 저장됐으면 좋겠습니다.. 햄버거집에 다대기 빼달라고 ...,4,0,0.438990
312,"카테고리 세부수정이 필요해 보입니다. ""반찬"" 검색하면 반찬전문점이 상위노출이 되어...",4,0,0.438445
977,잘쓰고 있습니다. 그런데 카드로 결재하다가 베인페이로 결재방법 바꿀려는데 변경만누르...,3,0,0.432003
504,얼마전 업데이트한후부터 앱이 실행이 아얘안되서 못시켜먹고있거든요;; 되다 안되다해요...,1,0,0.423744


In [81]:
# 각 토픽별로 확률이 가장 높은 5개문장을 한번에 프린트해 주세요 -반복문

# df3 = pd.DataFrame(df)

# topic_prob_0_top5 = df3[df3["topic_no"].apply(lambda x : x == 0)]#.sort_values(by=["topic_prob"],ascending=False).head(5)
# topic_prob_0_top5 = df3[df3["topic_no"].apply(lambda x : x == 0)]#.sort_values(by=["topic_prob"],ascending=False).head(5)
# print(topic_prob_0_top5)

n_topic = df["topic_no"].nunique()
print(f"{n_topic}개의 그룹이 있습니다.")

for n in range(n_topic):
    print(f"TOPIC : {n}")
    # df에서 topic이 n인 데이터만 추출한 수 topic_prob 기준으로 내림차순
    topic_df = df.loc[df["topic_no"]==n,:].sort_values(by=["topic_prob"],ascending=False)
    # topic_df의 text를 5개만 가져옴
    for i in range(5):
        print(topic_df["text"].values[i])
        print("-"*20)
    print("-"*50)

5개의 그룹이 있습니다.
TOPIC : 0
주문현황에서 실시간 지도 움직이는거 생성해주세요. 배민원만 실시간 지도 움직이는거만 보여주는거보다 배민원 없는지역에서는 일반배달대행 업체 기사 실시간 지도 움직이는거 생성해주세요. 가끔 음식 엎어지거나 식어서 도착알림 없는 사고 있는데, 일반배달대행 업체 기사 실시간 지도 움직임도 이젠 필수로 필요해요! 일반배달대행 업체 기사 불편불만 신고 기능 생성해주세요. 또 있어요. 일부가게에서 재료소진으로 인한 주문취소. 되는경우있는데, 일부메뉴 재료소진때문에 가게에서 전체메뉴 강제 취소 시키더라고요. 가게는 전화도 안하고 일방적으로 재료소진 시키는 사례 있어요. 재료소진으로 취소시 가게측에 재료소진 취소 사유 필수로해서 재료소진 메뉴이름 없으면 안된다는 문구 생성과동시에 꼭 재료소진된 메뉴 적으라는 가게측에 필수 기능생성 시키세요.
--------------------
최악. 배달이 잘못되도 앱이 어떤식으로 구성되는지 내 주문 건 못찾는다고 주인이 화를 내는걸 들어야 하고 나도 답답함. 프렌차이즈 주문이 잘못되서 앱에 나온 번호로 전화연결 하면 해당프렌차이즈 연결되는데 상담불가라고 배민상담 연결해줌. 그럼 또 꼬이고 꼬여서 계속 같은 설명 해야됨. 주문이 잘되냐 하면 그것도 아님. 한번은 b마트 물건 주문했는데 내가 구경도 안한 물품이 옴. 버그로 다른 물품이 골라져서 주문서에 있어서 그러려니 넘김. 주문자가 사용하는 앱도 가게주인이 사용하는 앱(사업장 프로그램)도 운영도 아주 많이 잘못되었음. 쓰다쓰다가 현타와서 앱 삭제함. 다시는 안쓸 예정.
--------------------
도로명 주소로 주소저장해도 배달만 시키면 지번주소로 바뀜. CS문의하니 우리는 문제없고 주문한 가게의 포스기가 원할하지 않다함. 도대체 입력된 정보를 자기맘대로 바꿔서 출력하는건 최신 AI인가요? 출력하는 기계가 정보를 변형해 출력한다는 말도 안되는 답변. 그럼 가게에서 출력하는 기계가 고사양 인공지능이 탑재된 포스기라는 소리 아닌가? 제일 엉망

In [ ]:
feature_names = count_vectorizer.get_feature_names_out()